In [1]:
%reload_ext autoreload
%autoreload 2

import optuna

c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


##### Hyperparameter Tuning with Accuracy Pruning


In [3]:
import models_hyper_optim_optuna
import models
import neural_net_model_train

c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project
X_train__78_features (1500, 78)
X_test__78_features (1500, 78)
Y_train_binary__78_features (1500,)
0.0    1500
dtype: int64
Y_test_binary (1500,)
classes_y:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
class_weights:  [1.0989011  1.03092784 0.85470085 1.02040816 0.99009901 1.0989011
 1.08695652 1.08695652 0.92592593 0.95238095 0.88495575 0.98039216
 1.11111111 0.98039216 0.99009901]

classes_class_weights: 
{0: 1.099,
 1: 1.031,
 2: 0.855,
 3: 1.02,
 4: 0.99,
 5: 1.099,
 6: 1.087,
 7: 1.087,
 8: 0.926,
 9: 0.952,
 10: 0.885,
 11: 0.98,
 12: 1.111,
 13: 0.98,
 14: 0.99}


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


DEVICE found:  cuda:0
number of classes:  15


In [4]:
channels = 1
batch_size= 32

args_model = {
    'number_of_classes':models_hyper_optim_optuna.number_of_classes, 
    'input_shape' : (batch_size, channels, models_hyper_optim_optuna.input_size),
    'use_gru_instead_of_lstm':True
    }

args_trial = {
    'max_epochs' : 7,
    'no_trials': 7, 
    'evaluation_function':'accuracy', 
    'USE_AUTOMATIC_MIXED_PRECISION':True
}

cnn_birnn_model = models.CnnBirnnModel(args_model)
models_trainning = neural_net_model_train.ModelsTrainning()

### Accuracy Tunning

In [5]:

from time import time


args_trial['evaluation_function']='accuracy'

start = time()


study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   cnn_birnn_model, models_trainning)

end = time()


[I 2022-09-27 10:08:31,047] A new study created in memory with name: no-name-4f6ec9c4-dbde-4759-80a0-498f537d123f
c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


NetModel(
  (cv1): Conv1d(1, 192, kernel_size=(3,), stride=(7,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(7,), padding=(0,))
  (bn1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(192, 190, kernel_size=(1,), stride=(6,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn2): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(190, 256, kernel_size=(1,), stride=(2,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(256, 92, kernel_size=(1,), stride=(2,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn4): BatchNorm1d(92, eps=1e-05, momentum=0.1, affine=

100%|██████████| 46/46 [00:01<00:00, 28.09it/s]



Train set: Average loss: 0.0705, Average accuracy: 2.7338%,
 	 Average f1_score: 0.0680, Average Area Under ROC: 0.5004 



100%|██████████| 46/46 [00:00<00:00, 65.77it/s]
[I 2022-09-27 10:08:47,569] Trial 0 finished with value: 0.0679347813129425 and parameters: {'n_layers': 5, 'n_units_l0': 192, 'no_strides_l0': 7, 'n_units_l1': 190, 'no_strides_l1': 6, 'n_units_l2': 256, 'no_strides_l2': 2, 'n_units_l3': 92, 'no_strides_l3': 2, 'n_units_l4': 228, 'no_strides_l4': 4, 'dropout_l4': 0.364452997164766, 'learning_rate': 0.0008689268130032155, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.0679347813129425.


...

100%|██████████| 46/46 [00:00<00:00, 66.98it/s]
[I 2022-09-27 10:09:32,976] Trial 4 finished with value: 0.06929347664117813 and parameters: {'n_layers': 5, 'n_units_l0': 64, 'no_strides_l0': 4, 'n_units_l1': 112, 'no_strides_l1': 6, 'n_units_l2': 170, 'no_strides_l2': 7, 'n_units_l3': 217, 'no_strides_l3': 1, 'n_units_l4': 145, 'no_strides_l4': 5, 'dropout_l4': 0.11598685275598744, 'learning_rate': 0.08635850132514833, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.07404891401529312.



Test set: Average loss: 32.3193, Average accuracy: 0.0693%,
 	  Average f1_score: 0.0598, Average Area Under ROC: 0.5181 

NetModel(
  (cv1): Conv1d(1, 210, kernel_size=(3,), stride=(6,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(6,), padding=(0,))
  (bn1): BatchNorm1d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(210, 237, kernel_size=(1,), stride=(1,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn2): BatchNorm1d(237, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(237, 120, kernel_size=(1,), stride=(7,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(7,), padding=(0,))
  (bn3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (bi_gru): GRU(120, 240, num_layers=2, batch_first=True, dropout=0

100%|██████████| 46/46 [00:00<00:00, 58.86it/s]
[I 2022-09-27 10:09:36,392] Trial 5 pruned. 


NetModel(
  (cv1): Conv1d(1, 190, kernel_size=(3,), stride=(5,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(5,), padding=(0,))
  (bn1): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(190, 68, kernel_size=(3,), stride=(5,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn2): BatchNorm1d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(68, 130, kernel_size=(1,), stride=(1,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (bn3): BatchNorm1d(130, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(130, 47, kernel_size=(1,), stride=(2,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn4): BatchNorm1d(47, eps=1e-05, momentum=0.1, affine=Tru

100%|██████████| 46/46 [00:01<00:00, 35.91it/s]
[I 2022-09-27 10:09:41,752] Trial 6 pruned. 


In [6]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  3
n_layers: 6
n_units_l0: 214
no_strides_l0: 2
n_units_l1: 176
no_strides_l1: 4
n_units_l2: 66
no_strides_l2: 5
n_units_l3: 123
no_strides_l3: 4
n_units_l4: 237
no_strides_l4: 4
n_units_l5: 165
no_strides_l5: 1
dropout_l5: 0.4937404375268971
learning_rate: 0.017015213386066333
optimizer: RMSprop


### Tuning Visualization

In [7]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [8]:
optuna.visualization.plot_optimization_history(study)

In [9]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-27 10:09:43,886] Your study has only completed trials with missing parameters.


In [10]:
optuna.visualization.plot_param_importances(study)

In [11]:
optuna.visualization.plot_slice(study)


### F1 score Tunning

In [12]:

args_trial['evaluation_function']='f1_score'

study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.HyperbandPruner(min_resource=3))

models_hyper_optim_optuna.start_optimize_objective(study, args_trial,
                                                   cnn_birnn_model, models_trainning)


[I 2022-09-27 10:09:45,309] A new study created in memory with name: no-name-152f8332-4025-42c0-b206-974f8ede6319
c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning:

Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.



NetModel(
  (cv1): Conv1d(1, 238, kernel_size=(3,), stride=(3,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (bn1): BatchNorm1d(238, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(238, 181, kernel_size=(3,), stride=(6,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn2): BatchNorm1d(181, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(181, 146, kernel_size=(1,), stride=(2,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn3): BatchNorm1d(146, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (bi_gru): GRU(146, 292, num_layers=2, batch_first=True, dropout=0.24106857592363595, bidirectional=True)
  (fc_output): Linear(in_features=584, out_features=32, bias=True)
)


100%|██████████| 46/46 [00:01<00:00, 32.43it/s]



Train set: Average loss: 0.0665, Average accuracy: 3.1537%,
 	 Average f1_score: 0.0625, Average Area Under ROC: 0.5037 



100%|██████████| 46/46 [00:00<00:00, 71.98it/s]
[I 2022-09-27 10:09:56,546] Trial 0 finished with value: 0.060973238199949265 and parameters: {'n_layers': 3, 'n_units_l0': 238, 'no_strides_l0': 3, 'n_units_l1': 181, 'no_strides_l1': 6, 'n_units_l2': 146, 'no_strides_l2': 2, 'dropout_l2': 0.24106857592363595, 'learning_rate': 0.028064523967701743, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.060973238199949265.


...

100%|██████████| 46/46 [00:01<00:00, 37.61it/s]
[I 2022-09-27 10:10:32,778] Trial 4 pruned. 


NetModel(
  (cv1): Conv1d(1, 199, kernel_size=(3,), stride=(4,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(4,), padding=(0,))
  (bn1): BatchNorm1d(199, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(199, 118, kernel_size=(3,), stride=(6,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn2): BatchNorm1d(118, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(118, 228, kernel_size=(1,), stride=(4,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(4,), padding=(0,))
  (bn3): BatchNorm1d(228, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(228, 149, kernel_size=(1,), stride=(2,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn4): BatchNorm1d(149, eps=1e-05, momentum=0.1, affin

100%|██████████| 46/46 [00:01<00:00, 34.52it/s]
[I 2022-09-27 10:10:38,454] Trial 5 pruned. 


NetModel(
  (cv1): Conv1d(1, 73, kernel_size=(3,), stride=(6,), bias=False)
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(6,), padding=(0,))
  (bn1): BatchNorm1d(73, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(73, 108, kernel_size=(1,), stride=(5,), bias=False)
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(5,), padding=(0,))
  (bn2): BatchNorm1d(108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(108, 219, kernel_size=(1,), stride=(2,), bias=False)
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn3): BatchNorm1d(219, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(219, 249, kernel_size=(1,), stride=(6,), bias=False)
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn4): BatchNorm1d(249, eps=1e-05, momentum=0.1, affine=T

100%|██████████| 46/46 [00:01<00:00, 28.08it/s]
[I 2022-09-27 10:10:45,238] Trial 6 pruned. 


In [13]:
trials_finished = study.get_trials()
print('number of trials finished: ', len(trials_finished),'\n')

print('number of the best trial: ',study.best_trial.number)

best_trial = study.best_trial
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

number of trials finished:  7 

number of the best trial:  2
n_layers: 7
n_units_l0: 121
no_strides_l0: 5
n_units_l1: 94
no_strides_l1: 7
n_units_l2: 133
no_strides_l2: 7
n_units_l3: 143
no_strides_l3: 3
n_units_l4: 83
no_strides_l4: 1
n_units_l5: 243
no_strides_l5: 2
n_units_l6: 163
no_strides_l6: 1
dropout_l6: 0.3355660099569828
learning_rate: 0.04984413890753318
optimizer: SGD


### Tuning Visualization

In [14]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [15]:
optuna.visualization.plot_optimization_history(study)

In [16]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-27 10:10:46,198] Your study has only completed trials with missing parameters.


In [17]:
optuna.visualization.plot_param_importances(study)

In [18]:
optuna.visualization.plot_slice(study)